## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_to_database(input_data):
    try:
        df = pd.read_csv(input_data)
        engine = sqlite3.connect("swimmers_data.db")
        df.to_sql("swimmers", con=engine, index = False)
        engine.close()
        print("Table successfully added to the database")
    except Exception as e:
        print(f"Error: {e}")

data = "/content/swimmers_stats.csv"
add_to_database(data)

Table successfully added to the database


In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///swimmers_data.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM swimmers
LIMIT 5;

 * sqlite:///swimmers_data.db
Done.


name,nationality,specialty,olympic_gold,olympic_silver,olympic_bronze,world_championship_gold,world_championship_silver,world_championship_bronze,total_medals
Michael Phelps,USA,Freestyle,23,3,3,26,6,2,63
Mark Spitz,USA,Freestyle/Butterfly,9,7,1,6,5,3,31
Jenny Thompson,USA,Freestyle,4,3,7,7,4,7,32
Matt Biondi,USA,Freestyle,5,4,1,8,3,1,22
Ian Thorpe,AUS,Freestyle,5,3,1,6,3,2,20


In [7]:
%%sql
PRAGMA table_info(swimmers);

 * sqlite:///swimmers_data.db
Done.


cid,name,type,notnull,dflt_value,pk
0,name,TEXT,0,None,0
1,nationality,TEXT,0,None,0
2,specialty,TEXT,0,None,0
3,olympic_gold,INTEGER,0,None,0
4,olympic_silver,INTEGER,0,None,0
5,olympic_bronze,INTEGER,0,None,0
6,world_championship_gold,INTEGER,0,None,0
7,world_championship_silver,INTEGER,0,None,0
8,world_championship_bronze,INTEGER,0,None,0
9,total_medals,INTEGER,0,None,0


In [8]:
%%sql
SELECT COUNT(*) AS row_count
FROM swimmers;

 * sqlite:///swimmers_data.db
Done.


row_count
28


## Question 2: **Answer**

In [9]:
%%sql
-- Retrieve Russian swimmers with at least 5 olympic medals
WITH russian_top_swimmers AS (
    SELECT
        name,
        nationality,
        olympic_gold,
        total_medals
    FROM swimmers
    WHERE nationality = 'RUS'
          AND olympic_gold >= 5
),
-- CTE to get swimmer with highest medal count
top_swimmer AS (
    SELECT
        name,
        nationality,
        MAX(total_medals) AS highest_total_medals
    FROM russian_top_swimmers
)
-- Main query to return four columns
SELECT
    rs.name,
    rs.nationality,
    rs.olympic_gold,
    rs.total_medals
FROM russian_top_swimmers rs
INNER JOIN top_swimmer tp
ON rs.total_medals = tp.highest_total_medals;

 * sqlite:///swimmers_data.db
Done.


name,nationality,olympic_gold,total_medals
Svetlana Romashina,RUS,6,15


## Question 3: **Answer**

In [10]:
%%sql
-- Calculate medals and create rank based total
WITH medals_per_nation AS (
    SELECT
        nationality,
        SUM(olympic_gold) AS total_olympic_gold,
        SUM(world_championship_gold) AS total_world_championship_gold,
        SUM(olympic_gold + world_championship_gold) AS combined_total_gold,
        ROW_NUMBER() OVER (ORDER BY SUM(olympic_gold + world_championship_gold) DESC)
        AS rank
    FROM swimmers
    GROUP BY nationality
)
-- Main query to return the results
SELECT
    nationality,
    total_olympic_gold AS olympic_gold,
    total_world_championship_gold AS world_championship_gold,
    combined_total_gold,
    rank
FROM medals_per_nation
WHERE rank <= 3
ORDER BY rank;

 * sqlite:///swimmers_data.db
Done.


nationality,olympic_gold,world_championship_gold,combined_total_gold,rank
USA,63,89,152,1
RUS,25,38,63,2
GDR,13,14,27,3


## Question 4: **Answer**

In [11]:
%%sql
-- Creating rows for categories
WITH categories(category) AS (
  SELECT 'Olympic Gold'
  UNION ALL
  SELECT 'Olympic Silver'
  UNION ALL
  SELECT 'Olympic Bronze'
  UNION ALL
  SELECT 'World Championship Gold'
  UNION ALL
  SELECT 'World Championship Silver'
  UNION ALL
  SELECT 'World Championship Bronze'
),
-- Adding values to rows from swimmers table
micheal_phelps_medals AS (
  SELECT
    c.category,
    CASE c.category
      WHEN 'Olympic Gold' THEN olympic_gold
      WHEN 'Olympic Silver' THEN olympic_silver
      WHEN 'Olympic Bronze' THEN olympic_bronze
      WHEN 'World Championship Gold' THEN world_championship_gold
      WHEN 'World Championship Silver' THEN world_championship_silver
      WHEN 'World Championship Bronze' THEN world_championship_bronze
    END AS total_medals
  FROM swimmers
  CROSS JOIN categories c
  WHERE name = 'Michael Phelps'
)
-- Get the category with the maximum medals
SELECT
    category,
    printf('%20s',total_medals) AS total_medals
FROM micheal_phelps_medals
WHERE total_medals = (SELECT MAX(total_medals) FROM micheal_phelps_medals);

 * sqlite:///swimmers_data.db
Done.


category,total_medals
World Championship Gold,26


##  Question 5: **Answer**

In [12]:
%%sql
SELECT
    aus_medals AS top_athlete_medals_aus,
    usa_medals AS top_athlete_medals_usa,
    (usa_medals * 100.0 / NULLIF(aus_medals, 0))
    AS percentage_us_to_aus
FROM (
    -- Subquery for Australia's top swimmer
    SELECT
        CAST(MAX(total_medals) AS REAL) AS aus_medals
    FROM swimmers
    WHERE nationality = 'AUS'
), (
    -- Subquery for USA's top swimmer
    SELECT
        CAST(MAX(total_medals) AS REAL) AS usa_medals
    FROM swimmers
    WHERE nationality = 'USA'
);

 * sqlite:///swimmers_data.db
Done.


top_athlete_medals_aus,top_athlete_medals_usa,percentage_us_to_aus
20.0,63.0,315.0


## Question 6: **Answer**

In [13]:
%%sql
SELECT
    CASE
        WHEN EXISTS(SELECT 1 FROM swimmers WHERE nationality = "IND")THEN
        'India is in the table'
        ELSE 'India is not in the table'
    END AS Indian_swimmers;

 * sqlite:///swimmers_data.db
Done.


Indian_swimmers
India is not in the table


In [14]:
%%sql
SELECT
    CASE
        WHEN 'IND' IN (SELECT nationality FROM swimmers) THEN
        'India is in the table'
        ELSE 'India is not in the table'
    END AS Indian_swimmers;

 * sqlite:///swimmers_data.db
Done.


Indian_swimmers
India is not in the table


##  Question 7: **Answer**

In [15]:
%%sql
-- Rank swimmers within each country based on Olympic gold medals
WITH ranked_swimmers AS (
    SELECT
        name,
        nationality,
        olympic_gold,
        RANK() OVER (PARTITION BY nationality ORDER BY olympic_gold DESC)
        AS rank
    FROM swimmers
)
-- Retrieve the top-ranked swimmer(s) from each country
SELECT
    name,
    nationality,
    olympic_gold
FROM ranked_swimmers
WHERE rank = 1
ORDER BY olympic_gold DESC;

 * sqlite:///swimmers_data.db
Done.


name,nationality,olympic_gold
Michael Phelps,USA,23
Kornelia Ender,GDR,7
Svetlana Romashina,RUS,6
Ian Thorpe,AUS,5
Wu Minxia,CHN,5
Gunnar Nielsen Aaby,DEN,5
Anita Lates,ITA,5
Kosuke Kitajima,JPN,4
Vladimir Salnikov,URS,4
Katinka Hosszu,HUN,3


In [16]:
# %%sql
# DROP TABLE swimmers;